In [26]:
import json

In [27]:
guesses_file = "../data/allowed_guesses.txt"
with open(guesses_file, "r") as fhand:
    actions = [word.strip() for word in fhand]
    actions = tuple(actions)
    
    
# Load the JSON data
with open("../data/precompute.json", "r") as file:
    colouring_data = json.load(file)


def get_transition_info(state, action):
    new_state = {}
    for word in state:
        colouring = colouring_data[action][word] 
    
        if colouring not in new_state:
            new_state[colouring] = [word]
        else:
            new_state[colouring].append(word)
        
    transition_info = {tuple(states): (len(states) / len(state)) for states in new_state.values()}

    
    return transition_info


In [28]:
state = ("bacon", "baggy", "banjo", "baron", "bawdy", "bayou", "cabby", "cabin", "cacao", "caddy", "cairn", "candy", "canny", "canon", "cargo", "carry", "daddy", "dairy", "dandy", "fairy", "fancy", "fanny", "fauna", "favor", "gamma", "gaudy", "gawky", "hairy", "handy", "happy", "hardy", "harpy", "harry", "havoc", "jazzy", "kappa", "karma", "kayak", "macaw", "macho", "macro", "madam", "mafia", "magic", "magma", "major", "mambo", "mamma", "mammy", "manga", "mango", "mangy", "mania", "manic", "manor", "march", "marry", "maxim", "mayor", "nadir", "nanny", "paddy", "pagan", "panic", "parka", "parry", "rabbi", "rabid", "radar", "radii", "radio", "rainy", "rajah", "ranch", "randy", "rapid", "rayon", "razor", "vapid", "vapor", "wacky", "wagon")


In [29]:
def get_transition_info(state : list, action : str):# -> dict[tuple, float]:# -> dict[tuple, float]:
    new_state = {}
    for word in state:
        colouring = colouring_data[action][word] 
    
        if colouring not in new_state:
            new_state[colouring] = [word]
        else:
            new_state[colouring].append(word)
        
    transition_info = {tuple(states): (len(states) / len(state)) for states in new_state.values()}

    
    return transition_info


In [30]:
state = tuple(["skate", "stage", "stake", "stare", "state", "stave", "stead", "steak", "steam"])
action = "skate"
get_transition_info(state, action)

{('skate',): 0.1111111111111111,
 ('stage', 'stare', 'stave'): 0.3333333333333333,
 ('stake',): 0.1111111111111111,
 ('state',): 0.1111111111111111,
 ('stead', 'steam'): 0.2222222222222222,
 ('steak',): 0.1111111111111111}

In [31]:
def value_function(t, state, actions, v_mem={}):# -> Any | tuple[float, None] | tuple[Literal[1], Any] | tuple...:
    if (t, state) in v_mem:
        return v_mem[(t, state)]
    
    if t >= 6 or (t == 5 and len(state) > 1):
        return (float("inf"), None)
    if t == 5 or len(state) == 1:
        return (1, state)
    elif len(state) == 2:
        return (1.5, state[0])

    state_value = float('inf')
    best_word = None

    combined = state + actions
    
    for action in combined:
        if t == 1:
            print(f"Searching for best word: {combined.index(action) / len(combined): .3%}, Current word = {action}")
    
        
        transition_info = get_transition_info(state, action)
        temp = 1
        
        if len(transition_info.keys()) == 1 and tuple(transition_info.keys())[0] == state:
            continue
        
        temp += (2 * len(state) - 1) / len(state)

        for st1 in transition_info.keys():
            
            if temp >= state_value:
                break
            elif st1 == action:
                continue
            
            res, _ = value_function(t + 1, state=st1, actions=actions)
            temp += transition_info[st1] * res

        if temp < state_value:
            state_value = temp
            best_word = action
    
    v_mem[(t, state)] = (state_value, best_word)
    return state_value, best_word

In [32]:
import os
import pickle

In [33]:
def load_v_mem(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'rb') as file:
            v_mem = pickle.load(file)
        return v_mem
    else:
        print(f"The file {file_path} does not exist.")
        return {}

In [34]:
def execute_and_save(t, state, actions, v_mem):
    result = value_function(t, state, actions, v_mem)
    
    # Save v_mem to a file
    with open('v_mem.pkl', 'wb') as file:
        pickle.dump(v_mem, file)
    
    return result

In [35]:
def process_actions(t, state, actions, v_mem):
    execute_and_save(t, state, actions, v_mem)

In [36]:
file_path = 'v_mem.pkl'
v_mem = load_v_mem(file_path)
state = ("belch", "belie", "belle", "belly", "below", "bilge", "bulge", "cello", "delve", "felon", "helix", "hello", "jelly", "melon", "relic", "welch")
process_actions(1, state, actions, v_mem)

The file v_mem.pkl does not exist.
Searching for best word:  0.000%, Current word = belch
Searching for best word:  0.007%, Current word = belie
Searching for best word:  0.013%, Current word = belle
Searching for best word:  0.020%, Current word = belly
Searching for best word:  0.027%, Current word = below
Searching for best word:  0.034%, Current word = bilge
Searching for best word:  0.040%, Current word = bulge
Searching for best word:  0.047%, Current word = cello
Searching for best word:  0.054%, Current word = delve
Searching for best word:  0.061%, Current word = felon
Searching for best word:  0.067%, Current word = helix
Searching for best word:  0.074%, Current word = hello
Searching for best word:  0.081%, Current word = jelly
Searching for best word:  0.087%, Current word = melon
Searching for best word:  0.094%, Current word = relic
Searching for best word:  0.101%, Current word = welch
Searching for best word:  0.108%, Current word = aahed
Searching for best word:  0.11

KeyboardInterrupt: 

In [38]:
score, word = v_mem[(1,(state))]
print(f"Best word: {word} ({score})")

KeyError: (1, ('belch', 'belie', 'belle', 'belly', 'below', 'bilge', 'bulge', 'cello', 'delve', 'felon', 'helix', 'hello', 'jelly', 'melon', 'relic', 'welch'))

In [ ]:
get_transition_info(state, 'crags')

{('abyss', 'amiss'): 0.11764705882352941,
 ('amass',): 0.058823529411764705,
 ('arson',): 0.058823529411764705,
 ('assay',): 0.058823529411764705,
 ('awash', 'quash', 'quasi'): 0.17647058823529413,
 ('brash',): 0.058823529411764705,
 ('brass',): 0.058823529411764705,
 ('chaos',): 0.058823529411764705,
 ('chasm',): 0.058823529411764705,
 ('crash',): 0.058823529411764705,
 ('crass',): 0.058823529411764705,
 ('gnash',): 0.058823529411764705,
 ('grasp',): 0.058823529411764705,
 ('grass',): 0.058823529411764705}

In [ ]:
get_transition_info(state, 'abyss')

{('abyss',): 0.058823529411764705,
 ('amass', 'amiss'): 0.11764705882352941,
 ('arson',): 0.058823529411764705,
 ('assay',): 0.058823529411764705,
 ('awash',): 0.058823529411764705,
 ('brash',): 0.058823529411764705,
 ('brass',): 0.058823529411764705,
 ('chaos',): 0.058823529411764705,
 ('chasm', 'crash', 'gnash', 'grasp', 'quash', 'quasi'): 0.35294117647058826,
 ('crass', 'grass'): 0.11764705882352941}

In [ ]:
get_transition_info(state, 'abuse')

{('skate', 'stage', 'stake', 'stare', 'state', 'stave'): 0.6666666666666666,
 ('stead', 'steak', 'steam'): 0.3333333333333333}